In [1]:
# !pip install tushare

     |████████████████████████████████| 174kB 4.9MB/s 
     |████████████████████████████████| 81kB 26.5MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/1a/9e/9d75d1da33706bb22e812e81b58716b49ea731f43dfa73f5c2
  Stored in directory: /root/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
Successfully built tushare simplejson


In [2]:
# !pip install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.1.1)


In [3]:
# !pip install tushare --upgrade

Requirement already up-to-date: tushare in /usr/local/lib/python3.6/dist-packages (1.2.36)


In [27]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA
import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.externals import joblib
import warnings
import tushare as ts
warnings.filterwarnings("ignore")

In [28]:
pro = ts.pro_api('52119551ebc32c734a42fd61ccf33344fde3c6195b0aacec58d15140')

# 特征处理

In [29]:
colums=['801010.SI', '801020.SI', '801030.SI','801040.SI', '801050.SI', '801080.SI', '801110.SI', '801120.SI',
        '801130.SI', '801140.SI', '801150.SI', '801160.SI', '801170.SI', '801180.SI', '801200.SI', '801210.SI',
        '801230.SI', '801250.SI', '801260.SI', '801270.SI', '801280.SI', '801300.SI', '801710.SI', '801720.SI',
        '801730.SI', '801740.SI', '801750.SI', '801760.SI', '801770.SI', '801780.SI', '801790.SI', '801880.SI',
        '801890.SI', '802600.SI']
df_pr = pd.DataFrame()
df = pd.DataFrame()
for arr in colums:
    df2=pro.sw_daily(ts_code=arr, start_date='20130101', end_date='20190626')
    df2=df2.sort_values(by="trade_date",ascending= True)
    df = pd.concat([df, df2])
df.loc[df['pct_change']>0,'y']=1
df.loc[df['pct_change']<0,'y']=0

In [30]:
df.head()

,ts_code,trade_date,name,open,low,high,close,change,pct_change,vol,amount,pe,pb,y
999,801010.SI,20150204,农林牧渔,2405.68,2383.01,2406.61,2384.64,-20.68,-0.86,65282.0,733331.0,57.79,3.63,0.0
998,801010.SI,20150205,农林牧渔,2408.53,2369.49,2415.44,2370.47,-14.16,-0.59,75076.0,846512.0,57.45,3.61,0.0
997,801010.SI,20150206,农林牧渔,2366.41,2320.88,2367.78,2330.24,-40.23,-1.70,76395.0,796091.0,56.48,3.55,0.0
996,801010.SI,20150209,农林牧渔,2328.82,2316.95,2335.93,2321.64,-8.60,-0.37,47210.0,525855.0,56.27,3.54,0.0
995,801010.SI,20150210,农林牧渔,2320.43,2317.66,2332.02,2332.02,10.38,0.45,38513.0,419652.0,56.52,3.55,1.0


In [31]:
x = []
y = []
for arr in colums:
    df1=df.loc[df['ts_code']==arr]
    df1=df1[['ts_code','trade_date','name','low','high','close','pct_change','vol','y']]
    df1['vol_change'] = df1['vol'].pct_change()
    df1['low_change'] = df1['low'].pct_change()
    df1['high_change'] = df1['high'].pct_change()
    df1['gaodi']=(df1['high']-df1['low'])/df1['low']
    df1['gaoclo']=(df1['high']-df1['close'])/(df1['high']-df1['low'])
    df1['diclo']=(df1['close']-df1['low'])/(df1['high']-df1['low'])
    df1['pct_change']=df1['pct_change']/10
    df1=df1[['pct_change','vol_change','high_change','low_change','gaodi','gaoclo','diclo','y']]
    df1= df1.dropna()
    dataset=df1.values
    for i in range(len(dataset)-17):
        x.append(dataset[i:(i+15),:8].tolist())
        y.append(dataset[(i+15):(i+16),7,np.newaxis].tolist())
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.1, random_state=30)
xtrain, xtest, ytrain, ytest = np.array(xtrain),np.array(xtest),np.array(ytrain),np.array(ytest)
# xtrain = np.reshape(xtrain, (xtrain.shape[0], xtrain.shape[2]))
# xtest = np.reshape(xtest, (xtest.shape[0], xtest.shape[2]))
ytrain = np.reshape(ytrain, (ytrain.shape[0], ytrain.shape[1]))
ytest= np.reshape(ytest, (ytest.shape[0], ytest.shape[1]))


In [32]:
ytrain.shape

(29935, 1)

# LSTM

In [33]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Activation
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from keras import layers
from sklearn.externals import joblib

In [0]:
# # 行业名字和行业代码的对应
# df_test4=
# df_me=pd.DataFrame(df_test4[['ts_code','name']])
# df_me=df_me.drop_duplicates()
# df_me
# df8=df_me[['name']]
# df8=df8.T
# df8.values

array([['农林牧渔', '采掘', '化工', '钢铁', '有色金属', '电子', '家用电器', '食品饮料', '纺织服装',
        '轻工制造', '医药生物', '公用事业', '交通运输', '房地产', '商业贸易', '休闲服务', '综合',
        '申万制造', '申万消费', '申万投资', '申万服务', '申万300指数', '建筑材料', '建筑装饰',
        '电气设备', '国防军工', '计算机', '传媒', '通信', '银行', '非银金融', '汽车', '机械设备',
        '交银装备']], dtype=object)

In [34]:
kf = KFold(n_splits=5, shuffle=True, random_state=520)


In [35]:
i=0
df_yanz=pd.DataFrame()
df_jieguo=pd.DataFrame()
for train,test in kf.split(xtrain):
    a = []
    b= []
    X_train, X_valid= xtrain[train,:], xtrain[test,:]
    y_train, y_valid = ytrain[train,:], ytrain[test,:]
    print(i, 'fold')
    regressor1 = Sequential()
    regressor1.add(LSTM(units = 70, return_sequences = True, input_shape = (xtrain.shape[1], 8)))
    regressor1.add(Dropout(0.2))
    regressor1.add(LSTM(units = 70, return_sequences = True))
    regressor1.add(Dropout(0.2))
    regressor1.add(LSTM(units = 70,return_sequences = True))
    regressor1.add(Dropout(0.2))
    regressor1.add(LSTM(units = 70))
    regressor1.add(Dropout(0.2))
    regressor1.add(layers.Dense(1, activation='sigmoid'))
    regressor1.compile(loss='binary_crossentropy',
                  optimizer='adam', metrics=['accuracy'])
    regressor1.fit(X_train,y_train, epochs = 100, batch_size = 80)
    joblib.dump(regressor1, './1lstm%s.model' % str(i))
    predicted_stock_price1 = regressor1.predict(X_valid)
    fpr1, tpr1, thresholds1 = roc_curve(y_valid,predicted_stock_price1, pos_label=1)
    a=auc(fpr1, tpr1)
    print(a)
    predicted_stock_price2 = regressor1.predict(xtest)
    fpr2, tpr2, thresholds2 = roc_curve(ytest, predicted_stock_price2, pos_label=1)
    b=auc(fpr2, tpr2)
    print(b)
#     df_yanz=df_yanz+pd.DataFrame(predicted_stock_price2)
#     df_pr=pd.DataFrame()
#     for arr in colums:
#         df4 = df_final.loc[df_final['ts_code']==arr]
#         dataset4=df4.values
#         x1=dataset2[0:10,3:11].tolist()
#         x1=np.array(x1)
#         x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
#         ytest1=regressor1.predict(x1)
#         y_p=pd.DataFrame(ytest1)
#         y_p=y_p.T
#         y_p['ts_code']=arr
#         y_p['trade_date']=[20190605,20190606,20190607,20190610,20190611]
#     df_pr = pd.concat([df_pr,y_p])
# # df_pr=df_me.merge(df_pr,how='left',on='name')
#     df_pr.columns=['i','ts_code','trade_date']
#     df_pr=df_pr[['ts_code','trade_date','i']]
#     df_jieguo['ts_code']=df_pr['ts_code']
#     df_jieguo['trade_date']=df_pr['trade_date']
#     df_jieguo['i']=df_pr['i']
#     joblib.dump(regressor1,'./lstm%s.model'%str(i))
    i=i+1
    
    

0 fold
Epoch 1/100
23948/23948 [==============================] - 42s 2ms/step - loss: 0.6917 - acc: 0.5220
Epoch 2/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.6907 - acc: 0.5267
Epoch 3/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.6891 - acc: 0.5367
Epoch 4/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.6882 - acc: 0.5342
Epoch 5/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.6863 - acc: 0.5405
Epoch 6/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.6840 - acc: 0.5464
Epoch 7/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.6830 - acc: 0.5497
Epoch 8/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.6808 - acc: 0.5489
Epoch 9/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.6763 - acc: 0.5579
Epoch 10/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.6

23948/23948 [==============================] - 23s 973us/step - loss: 0.2325 - acc: 0.8990
Epoch 81/100
23948/23948 [==============================] - 23s 971us/step - loss: 0.2293 - acc: 0.8995
Epoch 82/100
23948/23948 [==============================] - 23s 975us/step - loss: 0.2260 - acc: 0.9027
Epoch 83/100
23948/23948 [==============================] - 23s 976us/step - loss: 0.2250 - acc: 0.9046
Epoch 84/100
23948/23948 [==============================] - 24s 988us/step - loss: 0.2159 - acc: 0.9073
Epoch 85/100
23948/23948 [==============================] - 23s 980us/step - loss: 0.2214 - acc: 0.9038
Epoch 86/100
23948/23948 [==============================] - 23s 974us/step - loss: 0.2096 - acc: 0.9111
Epoch 87/100
23948/23948 [==============================] - 23s 975us/step - loss: 0.2084 - acc: 0.9124
Epoch 88/100
23948/23948 [==============================] - 23s 976us/step - loss: 0.2108 - acc: 0.9110
Epoch 89/100
23948/23948 [==============================] - 23s 976us/step - 

23948/23948 [==============================] - 23s 961us/step - loss: 0.2863 - acc: 0.8720
Epoch 60/100
23948/23948 [==============================] - 23s 952us/step - loss: 0.2829 - acc: 0.8734
Epoch 61/100
23948/23948 [==============================] - 23s 958us/step - loss: 0.2768 - acc: 0.8782
Epoch 62/100
23948/23948 [==============================] - 23s 950us/step - loss: 0.2692 - acc: 0.8826
Epoch 63/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.2671 - acc: 0.8822
Epoch 64/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.2637 - acc: 0.8825
Epoch 65/100
23948/23948 [==============================] - 24s 1ms/step - loss: 0.2533 - acc: 0.8909
Epoch 66/100
23948/23948 [==============================] - 28s 1ms/step - loss: 0.2526 - acc: 0.8879: 0s - loss: 0.2527 - acc: 0.88
Epoch 67/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.2510 - acc: 0.8891
Epoch 68/100
23948/23948 [==============================

23948/23948 [==============================] - 27s 1ms/step - loss: 0.4291 - acc: 0.7869
Epoch 39/100
23948/23948 [==============================] - 30s 1ms/step - loss: 0.4188 - acc: 0.7920
Epoch 40/100
23948/23948 [==============================] - 31s 1ms/step - loss: 0.4130 - acc: 0.7996
Epoch 41/100
23948/23948 [==============================] - 33s 1ms/step - loss: 0.4022 - acc: 0.8032
Epoch 42/100
23948/23948 [==============================] - 31s 1ms/step - loss: 0.3946 - acc: 0.8083
Epoch 43/100
23948/23948 [==============================] - 33s 1ms/step - loss: 0.3876 - acc: 0.8153
Epoch 44/100
23948/23948 [==============================] - 29s 1ms/step - loss: 0.3795 - acc: 0.8172
Epoch 45/100
23948/23948 [==============================] - 30s 1ms/step - loss: 0.3721 - acc: 0.8232
Epoch 46/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.3632 - acc: 0.8277
Epoch 47/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.3624 - acc: 

23948/23948 [==============================] - 29s 1ms/step - loss: 0.6410 - acc: 0.6046
Epoch 19/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.6326 - acc: 0.6155
Epoch 20/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.6252 - acc: 0.6229
Epoch 21/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.6153 - acc: 0.6357
Epoch 22/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.6064 - acc: 0.6446
Epoch 23/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.5960 - acc: 0.6576
Epoch 24/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.5852 - acc: 0.6695
Epoch 25/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.5784 - acc: 0.6721
Epoch 26/100
23948/23948 [==============================] - 24s 1ms/step - loss: 0.5658 - acc: 0.6847
Epoch 27/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.5598 - acc: 

23948/23948 [==============================] - 24s 1ms/step - loss: 0.1878 - acc: 0.9214
Epoch 99/100
23948/23948 [==============================] - 25s 1ms/step - loss: 0.1850 - acc: 0.9250
Epoch 100/100
23948/23948 [==============================] - 23s 980us/step - loss: 0.1749 - acc: 0.9303
0.8324269669047124
0.826487474851652
4 fold
Epoch 1/100
23948/23948 [==============================] - 40s 2ms/step - loss: 0.6915 - acc: 0.5186
Epoch 2/100
23948/23948 [==============================] - 26s 1ms/step - loss: 0.6909 - acc: 0.5254
Epoch 3/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.6881 - acc: 0.5332
Epoch 4/100
23948/23948 [==============================] - 33s 1ms/step - loss: 0.6874 - acc: 0.5372
Epoch 5/100
23948/23948 [==============================] - 32s 1ms/step - loss: 0.6859 - acc: 0.5371
Epoch 6/100
23948/23948 [==============================] - 32s 1ms/step - loss: 0.6840 - acc: 0.5447
Epoch 7/100
23948/23948 [=============================

23948/23948 [==============================] - 28s 1ms/step - loss: 0.2248 - acc: 0.9034
Epoch 78/100
23948/23948 [==============================] - 28s 1ms/step - loss: 0.2191 - acc: 0.9068
Epoch 79/100
23948/23948 [==============================] - 29s 1ms/step - loss: 0.2171 - acc: 0.9078
Epoch 80/100
23948/23948 [==============================] - 28s 1ms/step - loss: 0.2105 - acc: 0.9097
Epoch 81/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.2153 - acc: 0.9095
Epoch 82/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.1998 - acc: 0.9162
Epoch 83/100
23948/23948 [==============================] - 28s 1ms/step - loss: 0.2020 - acc: 0.9151
Epoch 84/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.1992 - acc: 0.9175
Epoch 85/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.1971 - acc: 0.9192
Epoch 86/100
23948/23948 [==============================] - 27s 1ms/step - loss: 0.1906 - acc: 

In [17]:
regressor1=joblib.load('./1lstm0.model')
regressor2=joblib.load('./1lstm1.model')
regressor3=joblib.load('./1lstm2.model')
regressor4=joblib.load('./1lstm3.model')
regressor5=joblib.load('./1lstm4.model')
predicted_stock_price3 = regressor1.predict(xtest)
predicted_stock_price4 = regressor2.predict(xtest)
predicted_stock_price5 = regressor3.predict(xtest)
predicted_stock_price6= regressor4.predict(xtest)
predicted_stock_price7 = regressor5.predict(xtest)
y=(predicted_stock_price3+predicted_stock_price4+predicted_stock_price5+predicted_stock_price6+predicted_stock_price7)/5
fpr3, tpr3, thresholds3 = roc_curve(ytest, y, pos_label=1)
auc(fpr3, tpr3)



0.8783419486414135

In [0]:
df_pr=pd.DataFrame()
i=0
range_index = (df_test4.trade_date > 20190521)
df_final = df_test4[range_index]
for i in range(5):
    model=joblib.load('./lstm%s.model'%str(i))
    for arr in colums:
        df4 = df_final.loc[df_final['ts_code']==arr]
        dataset4=df4.values
        x1=dataset2[0:10,3:11].tolist()
        x1=np.array(x1)
        x1=np.reshape(x1, (1,x1.shape[0],x1.shape[1]))
        ytest1= model.predict(x1)
        y_p=pd.DataFrame(ytest1)
        y_p=y_p.T
        y_p['ts_code']=arr
        y_p['trade_date']=[20190605,20190606,20190607,20190610,20190611]
y_p
#     df_pr = pd.concat([df_pr,y_p])
# # df_pr=df_me.merge(df_pr,how='left',on='name')
#     df_pr.columns=['w','i','ts_code','trade_date']
#     df_pr=df_pr[['ts_code','trade_date','i']]
#     df_jieguo['ts_code']=df_pr['ts_code']
#     df_jieguo['trade_date']=df_pr['trade_date']
#     df_jieguo['i']=df_pr['i']
#     i=i+1

# 新闻

In [0]:
df1 = pd.read_csv(r'./TRAINSET_STOCK.csv')
df_add = pd.read_csv(r'./20190506_STOCK.csv')
df_test = pd.read_csv(r'./20190514_STOCK.csv')
df1 = pd.concat([df1,df_add,df_test])

In [0]:
df2=df1[['name','ts_code']]
df2 = df2.drop_duplicates(keep='first')


,0,1152,2304,3456,3656,4608,5760,6912,8064,9216,10368,11520,12672,13824,14976,16128,17280,17480,18432,19584,20736,21888,23040,24192,25344,26496,27648,28800,29952,31104,32256,33408,34560,35712,36864,38016
name,农林牧渔,采掘,化工,黑色金属,钢铁,有色金属,电子,家用电器,食品饮料,纺织服装,轻工制造,医药生物,公用事业,交通运输,房地产,商业贸易,餐饮旅游,休闲服务,综合,申万制造,申万消费,申万投资,申万服务,申万300指数,建筑材料,建筑装饰,电气设备,国防军工,计算机,传媒,通信,银行,非银金融,汽车,机械设备,交银装备
ts_code,801010,801020,801030,801040,801040,801050,801080,801110,801120,801130,801140,801150,801160,801170,801180,801200,801210,801210,801230,801250,801260,801270,801280,801300,801710,801720,801730,801740,801750,801760,801770,801780,801790,801880,801890,802600


In [0]:
df_news1 = pd.read_csv(r'./TRAINSET_NEWS.csv')
df_news2= pd.read_csv(r'./20190506_NEWS.csv')
df_news3= pd.read_csv(r'./20190514_NEWS.csv')
df_news = pd.concat([df_news1,df_news2,df_news3])
df_news.info

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27077 entries, 0 to 112
Data columns (total 4 columns):
id         27077 non-null object
date       27077 non-null int64
title      27048 non-null object
content    26915 non-null object
dtypes: int64(1), object(3)
memory usage: 1.0+ MB


# Tensorflow

In [0]:
import tensorflow as tf
time_step=10      #时间步
rnn_unit=100      #hidden layer units
batch_size=80     #每一批次训练多少个样例
input_size=3      #输入层维度
output_size=1     #输出层维度
lr=0.0006 


In [0]:
# 训练集
def get_train_data(batch_size=80,time_step=10,train_begin=0,train_end=900):
  for arr in columns:
    df1=df.loc[df['name']==arr]
    df1=df1[['ts_code','trade_date','name','low','high','pct_change','vol','y']]
    df1['vol_change'] = df['vol'].pct_change()
    df1['low_change'] = df['low'].pct_change()
    df1['high_change'] = df['high'].pct_change()
    df1=df1[['pct_change','vol_change','high_change','low_change','y']]
    df1= df1.dropna()
    dataset=df1.values
    batch_index=[]
    data_train=dataset[train_begin:train_end]
    normalized_train_data=(data_train-np.mean(data_train,axis=0))/np.std(data_train,axis=0)  #标准化
    train_x,train_y=[],[]   #训练集x和y初定义
    for i in range(len(normalized_train_data)-time_step):
        if i % batch_size==0:
            batch_index.append(i)
            x=normalized_train_data[i:i+time_step,:5]
            y=normalized_train_data[i:i+time_step,4,np.newaxis]
            train_x.append(x.tolist())
            train_y.append(y.tolist())
    batch_index.append((len(normalized_train_data)-time_step))
    return batch_index,train_x,train_y
 


In [0]:
# 测试集
def get_test_data(time_step=10,test_begin=900):
  for arr in columns:
    df1=df.loc[df['name']==arr]
    df1=df1[['ts_code','trade_date','name','low','high','pct_change','vol','y']]
    df1['vol_change'] = df['vol'].pct_change()
    df1['low_change'] = df['low'].pct_change()
    df1['high_change'] = df['high'].pct_change()
    df1=df1[['pct_change','vol_change','high_change','low_change','y']]
    df1= df1.dropna()
    dataset=df1.values
    data_test=dataset[test_begin:]
    mean=np.mean(data_test,axis=0)
    std=np.std(data_test,axis=0)
    normalized_test_data=(data_test-mean)/std  #标准化
    size=(len(normalized_test_data)+time_step-1)//time_step  #有size个sample 
    test_x,test_y=[],[]  
    for i in range(size-1):
        x=normalized_test_data[i*time_step:(i+1)*time_step,:5]
        y=normalized_test_data[i*time_step:(i+1)*time_step,4]
        test_x.append(x.tolist())
        test_y.extend(y.tolist())
    return mean,std,test_x,test_y

In [0]:
# 模型搭建
weights = {
    'in': tf.Variable(tf.random_normal([input_size, rnn_unit])),
    'out': tf.Variable(tf.random_normal([rnn_unit, 1 ]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[rnn_unit, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[1, ]))
}
def lstm(X):     
    time_step=10
    w_in=weights['in']
    b_in=biases['in']  
    input=tf.reshape(X,[-1,input_size])  #需要将tensor转成2维进行计算，计算后的结果作为隐藏层的输入
    input_rnn=tf.matmul(input,w_in)+b_in
    input_rnn=tf.reshape(input_rnn,[-1,time_step,rnn_unit])  #将tensor转成3维，作为lstm cell的输入
    cell=tf.nn.rnn_cell.BasicLSTMCell(rnn_unit)
    init_state=cell.zero_state(batch_size,dtype=tf.float32)
    output_rnn,final_states=tf.nn.dynamic_rnn(cell, input_rnn,initial_state=init_state, dtype=tf.float32)  #output_rnn是记录lstm每个输出节点的结果，final_states是最后一个cell的结果
    output=tf.reshape(output_rnn,[-1,rnn_unit]) #作为输出层的输入
    w_out=weights['out']
    b_out=biases['out']
    pred=tf.matmul(output,w_out)+b_out
    return pred,final_states

In [0]:
def train_lstm(batch_size=80,time_step=10,train_begin=0,train_end=30000):
    X=tf.placeholder(tf.float32, shape=[None,time_step,input_size])
    Y=tf.placeholder(tf.float32, shape=[None,time_step,output_size])
    batch_index,train_x,train_y=get_train_data(batch_size,time_step,train_begin,train_end)
    pred,_=lstm(X)
    #损失函数
    loss=tf.reduce_mean(tf.square(tf.reshape(pred,[-1])-tf.reshape(Y, [-1])))
    train_op=tf.train.AdamOptimizer(lr).minimize(loss)
    saver=tf.train.Saver(tf.global_variables(),max_to_keep=15)
    module_file = tf.train.latest_checkpoint()    
    with tf.Session() as sess:
        #sess.run(tf.global_variables_initializer())
        saver.restore(sess, module_file)
        #重复训练2000次
        for i in range(2000):
            for step in range(len(batch_index)-1):
                _,loss_=sess.run([train_op,loss],feed_dict={X:train_x[batch_index[step]:batch_index[step+1]],Y:train_y[batch_index[step]:batch_index[step+1]]})
            print(i,loss_)
            if i % 200==0:
                print("保存模型：",saver.save(sess,'stock2.model',global_step=i))

In [0]:
from __future__ import print_function
def layer(inputs,input_dim,output_dim,activation=None):
  with tf.name_scope('layer'):
    with tf.name_scope('weights'):
       W = tf.Variable(tf.random_normal([input_dim, output_dim]))
    with tf.name_scope('biases'):
       b = tf.Variable(tf.random_normal([1, output_dim])+0.1)
    with tf.name_scope('Wx_plus_b'):
       XWb = tf.matmul(inputs, W) + b
    if activation is None:
        outputs = XWb
    else:
        outputs = activation(XWb)
    return outputs


In [0]:
with tf.name_scope('inputs'):
  xs= tf.placeholder(tf.float32, [None,80],name='input_in')
  ys=tf.placeholder(tf.float32, [None,5],name='intput_out')
l1=layer(xs,80,100,activation=tf.nn.relu)
prediction = layer(l1, 100, 5, activation=tf.nn.sigmoid)
with tf.name_scope('loss'):
  loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - prediction),
                     reduction_indices=[1]))
with tf.name_scope('train'):
  train_step = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
init = tf.global_variables_initializer()
sess = tf.Session()
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:  # tensorflow version < 0.12
    writer = tf.train.SummaryWriter('logs/', sess.graph)
else: # tensorflow version >= 0.12
    writer = tf.summary.FileWriter("logs/", sess.graph)
if int((tf.__version__).split('.')[1]) < 12 and int((tf.__version__).split('.')[0]) < 1:
    init = tf.initialize_all_variables()
else:
    init = tf.global_variables_initializer()
sess.run(init)

In [0]:
for i in range(10000):
    # training
    sess.run(train_step, feed_dict={xs: xtrain, ys: ytrain})
    if i % 50 == 0:
        # to see the step improvement
        print(sess.run(loss, feed_dict={xs: xtrain, ys: ytrain}))

In [0]:
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.preprocessing import LabelBinarizer

In [0]:
keep_prob = tf.placeholder(tf.float32)
digits = load_digits()
X = digits.data
y = digits.target
y = LabelBinarizer().fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3)

In [0]:
y


array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0]])

In [0]:
sess.close()

In [0]:
with tf.Session() as sess:
  result2=sess.run(input1,feed_dict={input1:[7.]})
  print(result2)

[7.]


In [0]:
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [0]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [0]:
def compute_accuracy(v_xs, v_ys):
    global prediction
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result
xs = tf.placeholder(tf.float32, [None, 784])
ys = tf.placeholder(tf.float32, [None, 10])
prediction = layer(xs, 784, 10, activation=tf.nn.softmax)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
reduction_indices=[1])) 
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(2000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
  if i % 50 == 0:
        print(compute_accuracy(
            mnist.test.images, mnist.test.labels))

0.1156
0.6279
0.7325
0.7706
0.8001
0.8138
0.8277
0.8361
0.8416
0.8492
0.8513
0.8554
0.856
0.862
0.8628
0.8631
0.8694
0.8715
0.876
0.8738
0.8756
0.8785
0.8798
0.8803
0.8789
0.8821
0.8841
0.8811
0.8847
0.8878
0.8852
0.8894
0.8868
0.8903
0.8842
0.8878
0.8924
0.8934
0.8928
0.8919
